In [1]:
TRY1 = "fʹαr (fear) « homme », le génitif fʹirʹ (fir)"

In [4]:
FORM_MAPS = {
    "le génitif": "genetive",
    "gen.": "genetive",
    "dat.": "dative",
    "plur.": "plural"
}

In [7]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"

In [10]:
m = re.match(REGEX_CASE, TRY1)
m.groups()

('fʹαr', 'fear', 'homme', 'le génitif', 'fʹirʹ', 'fir')

In [ ]:
DATA = {}